<a href="https://colab.research.google.com/github/vimesh630/ML_CW/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries and Mount Google Drive

In [21]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
drive_path = '/content/drive/MyDrive/ML Coursework/'
file1_path = os.path.join(drive_path, 'bank+marketing/bank-additional/bank-additional/bank-additional-full.csv')
file2_path = os.path.join(drive_path, 'bank+marketing/bank-additional/bank-additional/bank-additional.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load and Merge Datasets

In [22]:
# Load the datasets
df1 = pd.read_csv(file1_path, sep=';')
df2 = pd.read_csv(file2_path, sep=';')

# Merge the datasets
data = pd.concat([df1, df2], axis=0, ignore_index=True)

# Verify the merged dataset
print("Columns after loading:")
print(data.columns)

Columns after loading:
Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')


Handle Missing Values

In [23]:
# Handle missing values for numerical and categorical columns
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Identify numerical and categorical columns
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Impute missing values for numerical columns
for col in numerical_cols:
    data[col] = numerical_imputer.fit_transform(data[[col]])

# Impute missing values for categorical columns
for col in categorical_cols:
    data[col] = categorical_imputer.fit_transform(data[[col]].values.reshape(-1, 1)).ravel()

# Verify no missing values remain
print("Missing values after imputation:")
print(data.isnull().sum())

Missing values after imputation:
age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


Ensure the 'y' Column Exists

In [24]:
# Verify that the 'y' column exists before processing
print("Columns before target conversion:")
print(data.columns)
print("First few rows of 'y':")
print(data['y'].head())

Columns before target conversion:
Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')
First few rows of 'y':
0    no
1    no
2    no
3    no
4    no
Name: y, dtype: object


Convert Target Variable to Binary

In [25]:
# Convert 'y' column to binary
data['y'] = data['y'].apply(lambda x: 1 if x == 'yes' else 0)

# Confirm the conversion
print("Target value counts after conversion:")
print(data['y'].value_counts())

Target value counts after conversion:
y
0    40216
1     5091
Name: count, dtype: int64


Encode Categorical Variables

In [26]:
# Encode categorical variables
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_encoded = pd.DataFrame(
    encoder.fit_transform(data[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols),
    index=data.index
)

# Drop original categorical columns and concatenate encoded columns
data = pd.concat([data.drop(columns=categorical_cols), categorical_encoded], axis=1)

# Verify encoding
print("Data after encoding:")
print(data.head())

Data after encoding:
    age  duration  campaign  pdays  previous  emp.var.rate  cons.price.idx  \
0  56.0     261.0       1.0  999.0       0.0           1.1          93.994   
1  57.0     149.0       1.0  999.0       0.0           1.1          93.994   
2  37.0     226.0       1.0  999.0       0.0           1.1          93.994   
3  40.0     151.0       1.0  999.0       0.0           1.1          93.994   
4  56.0     307.0       1.0  999.0       0.0           1.1          93.994   

   cons.conf.idx  euribor3m  nr.employed  ...  day_of_week_fri  \
0          -36.4      4.857       5191.0  ...              0.0   
1          -36.4      4.857       5191.0  ...              0.0   
2          -36.4      4.857       5191.0  ...              0.0   
3          -36.4      4.857       5191.0  ...              0.0   
4          -36.4      4.857       5191.0  ...              0.0   

   day_of_week_mon  day_of_week_thu  day_of_week_tue  day_of_week_wed  \
0              1.0              0.0     

Handle Class Imbalance with SMOTE

In [27]:
# Ensure all features are numeric
print("Feature dtypes before SMOTE:")
print(X.dtypes.value_counts())

# Select only numeric columns to pass to SMOTE
X = X.select_dtypes(include=['int64', 'float64'])

# Verify features after filtering
print("Feature shape after selecting numeric columns:", X.shape)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Verify SMOTE results
print("Shapes after SMOTE:")
print("Features:", X_resampled.shape)
print("Target:", y_resampled.shape)


Feature dtypes before SMOTE:
int64      5
float64    5
Name: count, dtype: int64
Feature shape after selecting numeric columns: (45307, 10)
Shapes after SMOTE:
Features: (80432, 10)
Target: (80432,)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Split Dataset into Training and Testing Sets

In [28]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

# Verify the split
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (64345, 10)
Testing set shape: (16087, 10)


Scale Features for Neural Networks

In [29]:
# Scale features for Neural Networks
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verify scaling
print("Scaled training set shape:", X_train_scaled.shape)
print("Scaled testing set shape:", X_test_scaled.shape)

Scaled training set shape: (64345, 10)
Scaled testing set shape: (16087, 10)


Save Files to Google Drive

In [31]:
# Create an output directory
output_dir = os.path.join(drive_path, 'Preprocessed Dataset')
os.makedirs(output_dir, exist_ok=True)

# Save files
pd.DataFrame(X_train).to_csv(os.path.join(output_dir, 'X_train.csv'), index=False)
pd.DataFrame(X_test).to_csv(os.path.join(output_dir, 'X_test.csv'), index=False)
pd.DataFrame(X_train_scaled).to_csv(os.path.join(output_dir, 'X_train_scaled.csv'), index=False)
pd.DataFrame(X_test_scaled).to_csv(os.path.join(output_dir, 'X_test_scaled.csv'), index=False)
pd.DataFrame(y_train).to_csv(os.path.join(output_dir, 'y_train.csv'), index=False)
pd.DataFrame(y_test).to_csv(os.path.join(output_dir, 'y_test.csv'), index=False)

print("Preprocessing complete. Files saved to Google Drive.")

Preprocessing complete. Files saved to Google Drive.
